# IMPORT LIBRARIES

In [1]:
!pip3 install numpy-financial
!pip3 install pandasql
!pip3 install scipy

import numpy as np
import pandas as pd

import numpy_financial as npf
from pandasql import sqldf
import math as mth
import scipy.stats
import statsmodels.api as sm
import math
from typing import Tuple, Dict, Sequence



import time

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=e22dfe8ee90f99fed606ebf3be4f997d14d8f2abb2bd691f6ab2a7668679963b
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [2]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

# LOAD DATA FROM GITHUB

In [8]:
# Rejection by Interest Rate
url_reject = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
b1fc825f2029dcea1d1db061cebd026a3b867ba6/data/rejection_data_final.csv"
rejection_df = pd.read_csv(url_reject)

# Default Probability
url_default = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
b1fc825f2029dcea1d1db061cebd026a3b867ba6/data/rating_default_prob_final.csv"
rating_defaults_df = pd.read_csv(url_default)

rating_default_dict = {}
for rating, grp in rating_defaults_df.groupby("rating"):
    default_vals  = grp["is_default"].astype(int).values
    default_probs = grp["probability"].astype(float).values
    default_probs = default_probs / default_probs.sum()
    rating_default_dict[rating] = (default_vals, default_probs)


# Default Month-on-book Probability
url_default_mob = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
b1fc825f2029dcea1d1db061cebd026a3b867ba6/data/default_mob_prob_final.csv"
default_mob_df = pd.read_csv(url_default_mob)


# Tenor Probability
url_tenor = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
02d62449a66b671512f6b84cdc6cc9343a2ac1c8/data/tenor_prob_final.csv"
tenor_df = pd.read_csv(url_tenor)


# LGD Probability
url_lgd = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
02d62449a66b671512f6b84cdc6cc9343a2ac1c8/data/lgd_final.csv"
lgd_df = pd.read_csv(url_lgd)


# Interest Rate Probability
url_interest = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
02d62449a66b671512f6b84cdc6cc9343a2ac1c8/data/interest_prob_final.csv"
interest_prob_df = pd.read_csv(url_interest)


# Application Rating Probability
url_prob_rating = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
1f75156d82b00b2d96352e8542360269fe0fc8a4/data/rating_prob_final.csv"
rating_prob_df = pd.read_csv(url_prob_rating)


# Principal Amount Probability
url_prin_prob = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
387da19fdcae157d5a40a33bca640e21dab2d965/data/prin_amt_prob_final.csv"
prin_prob_df = pd.read_csv(url_prin_prob)

# USER DEFINED-FUNCTIONS

## Acceptance Probability

In [205]:
def fit_logit_model_from_url(url
                             , target='is_rejected'
                             , feature='interest_rate'):
    data = pd.read_csv(url)

    features = [col for col in data.columns if col != target]
    X = data[features]
    y = data[target]

    X_const = sm.add_constant(X)

    logit_model = sm.Logit(y, X_const).fit(disp=0)

    intercept = logit_model.params['const']
    coef = logit_model.params[feature]

    return intercept, coef, logit_model

intercept, coef, logit_model = fit_logit_model_from_url(url_reject)

## Calculate Present Value

In [368]:
def calculate_present_val(prin_amt
                          , tenor
                          , eir
                          , default_mob
                          , recovery_rate
                          , payment_freq
                          , disc_rate
                          ):

    """
    Calculate the present value of loan cash flows and the loss amount.

    Parameters
    ----------
    prin_amt : float
        Principal amount of the loan.
    tenor : int
        Total number of payment periods.
    eir : float
        Effective annual interest rate.
    default_mob : int
        Default month-on-book (-1 indicates no default).
    recovery_rate : float
        Recovery rate applied to the remaining principal.
    payment_freq : int, optional
        Number of payments per year.
    disc_rate : float, optional
        Annual discount rate.

    Returns
    -------
    present_val : float
        Present value of the discounted cash flows (including recovery).
    loss_amt : float
        Loss amount computed as principal minus present value.
    """

    mob = np.arange(1, tenor + 1)

    #default_mob - 3 for is_not_default while int(default_mob) - 5 for cumulative
    #principal payment IS NOT a mistake.

    is_not_default = (mob < default_mob - 3) | (default_mob == -1)

    disc_factor = 1 / (1 + disc_rate / payment_freq) ** mob

    rate = eir / payment_freq

    prin_pmt = npf.ppmt(rate=rate, per=mob, nper=tenor, pv=-prin_amt)
    emi_pmt = np.sum(npf.pmt(rate=rate, nper=tenor, pv=-prin_amt) * is_not_default * disc_factor)

    cumul_prin_pmt = np.cumsum(prin_pmt)

    pv_recovery_amt = 0

    if default_mob >= 4:

        if default_mob == 4:
            remaining_prin = prin_amt
            pv_recovery_amt = remaining_prin * recovery_rate
        else:
            index = int(default_mob) - 5 #index of an array begins with 0.
            cumul_paid = cumul_prin_pmt[index]
            remaining_prin = prin_amt - cumul_paid
            pv_recovery_amt = remaining_prin * recovery_rate

    present_val = emi_pmt + pv_recovery_amt
    if prin_amt - present_val < 0:
        loss_amt = 0
    else:
        loss_amt = prin_amt - present_val

    return np.round(present_val, 0), np.round(loss_amt, 0)

## Default Probability by Rating

In [30]:
def is_default_func(rating):
    """
    Simulate default based on credit rating
    using precomputed CSV data.

    Parameters
    ----------
    rating : int
        Credit rating.

    Returns
    -------
    int
        Simulated default month-on-book.
    """
    try:
        default_vals, default_probs = rating_default_dict[rating]
    except KeyError:
        raise ValueError(f"No default data available for rating: {rating}")

    return np.random.choice(a=default_vals, p=default_probs)

# Vectorize the function to apply it elementwise.
default_vector_func = np.vectorize(is_default_func)

## Default Month-on-Book

In [31]:
def default_mob_func(is_default, tenor):
    """
    If is_default==0, return -1.
    If is_default==1, sample a default month‐on‐book ∈ [3, tenor],
    according to `default_mob_probs`, and return it.

    Parameters
    ----------
    is_default : {0, 1}
        0 ⇒ no default; 1 ⇒ default occurs.
    tenor : int
        Loan tenor in months (allowed values: 12, 24, 36, 48).

    Returns
    -------
    int
        -1 if no default, else sampled month ≤ tenor.
    """
    if is_default == 0:
        return -1

    # restrict to months ≤ tenor

    default_mob_vals = default_mob_df['default_mob'].values
    default_mob_probs = default_mob_df['probability'].values


    mask = default_mob_vals <= tenor
    possible = default_mob_vals[mask]
    probs    = default_mob_probs[mask]

    probs = probs / probs.sum()

    return int(np.random.choice(possible, p=probs))


default_mob_vector_func = np.vectorize(default_mob_func)

## Calculate Loss Given Default

In [19]:
def lgd_func(default_mob):
    """
    Simulate loss given default (LGD) based on
    credit rating and default month-on-book.

    Parameters
    ----------
    rating : int
        Loan credit rating.
    default_mob : int
        Default month-on-book.

    Returns
    -------
    float
        Loss given default (LGD).
        Returns -1.0 if no default (default_mob == -1).
    """
    if default_mob == -1:
        return -1

    lgd_vals = lgd_df['lgd'].values
    lgd_probs = lgd_df['probability'].values

    lgd_probs = lgd_probs / lgd_probs.sum()

    return np.random.choice(lgd_vals, p=lgd_probs)

lgd_vector_func = np.vectorize(lgd_func)

## Simulate New Loan Data

In [367]:
def simulate_new_loan(N_new
                      , ids_new
                      , rating_input_new
                      , prin_input_new
                      , tenor_input_new
                      , payment_freq
                      ):
    """
    Simulate new loan data including credit rating, principal amount
    , tenor,default month-on-book, loss given default
    , and recovery-based present value metrics.

    Parameters
    ----------
    N_new : int
        Number of new loans (scenarios) to simulate.
    ids_new : array-like
        Unique identifiers for each new loan.
    rating_input_new : int
        Fixed credit rating for all new loans.
    prin_input_new : float
        Fixed principal amount for all new loans.

    Returns
    -------
    pd.DataFrame
        DataFrame containing simulated loan data.
    """

    rating_new = np.full(N_new, rating_input_new)
    prin_amt_new = np.full(N_new, prin_input_new)
    tenor_new = np.full(N_new, tenor_input_new)

    default_new = default_vector_func(rating=rating_new)
    default_mob_new = default_mob_vector_func(is_default=default_new, tenor=tenor_new)
    lgd_new = lgd_vector_func(default_mob=default_mob_new)

    gen_loan_info_new = pd.DataFrame({
        'k_index': ids_new,
        'credit_rating': rating_new,
        'prin_amt': prin_amt_new,
        'tenor': tenor_new,
        'default_mob': default_mob_new,
        'lgd': lgd_new
    })

    gen_loan_info_new['recovery_rate'] = np.where(
        gen_loan_info_new['lgd'] == -1, -1,
        1 - gen_loan_info_new['lgd']
    )

    return gen_loan_info_new

## Simulate Existing Loan Data

In [386]:
def simulate_current_portfolio(N
                               , ids
                               , obs
                               , payment_freq
                               , disc_rate
                               , port_quality=None
                               ):
    """
    Simulate loan data including credit rating
    , principal amount, tenor, interest rate,
    default month-on-book, and loss given default.

    Parameters
    ----------
    N : int
        Number of loans to simulate.
    ids : array-like
        Unique loan IDs.
    risk_tolerance : float
        Fraction representing risk tolerance.
    obs : int
        Number of observations per k_index grouping.
    payment_freq : int
        Number of payments per year.
    disc_rate : float
        Annual discount rate.
    port_quality : str, optional
        Portfolio quality indicator.

    Returns
    -------
    pd.DataFrame
        DataFrame containing simulated loan information.
    """

    rating_vals = rating_prob_df["rating"].values
    rating_probs = rating_prob_df["probability"].values

    if port_quality == "g": #good portfolio
        mask = rating_vals <= 6
    # elif port_quality == "m": #medium portfolio
        # mask = (rating_vals >= 4) & (rating_vals <= 6)
    elif port_quality == "b": #bad portfolio
        mask = rating_vals >= 7
    else: #mixed
        mask = np.ones_like(rating_vals, dtype=bool)

    possible = rating_vals[mask]
    probs    = rating_probs[mask]

    rating_probs_m = probs / probs.sum()

    rating = np.random.choice(possible, size=N, p=rating_probs_m)

    prin_vals = prin_prob_df["principal_amount"].values
    prin_probs = prin_prob_df["probability"].values
    prin_probs = prin_probs / prin_probs.sum()
    prin_amt = np.random.choice(prin_vals, size=N, p=prin_probs)

    tenor_vals = tenor_df["tenor"].values
    tenor_probs = tenor_df["probability"].values
    tenor_probs = tenor_probs / tenor_probs.sum()
    tenor = np.random.choice(tenor_vals, size=N, p=tenor_probs)

    eir_vals = interest_prob_df["interest_rate"].values
    eir_probs = interest_prob_df["probability"].values
    eir_probs = eir_probs / eir_probs.sum()
    eir = np.random.choice(eir_vals, size=N, p=eir_probs)

    default = default_vector_func(rating=rating)

    default_mob = default_mob_vector_func(is_default=default, tenor=tenor)

    lgd = lgd_vector_func(default_mob=default_mob)

    gen_loan_info = pd.DataFrame({
        'loan_id': ids,
        'credit_rating': rating,
        'prin_amt': prin_amt,
        'tenor': tenor,
        'eir': eir,
        'default_mob': default_mob,
        'lgd': lgd
    })

    gen_loan_info['recovery_rate'] = np.where(
        gen_loan_info['lgd'] == -1,
        -1,
        1 - gen_loan_info['lgd']
    )

    pv_loss = [
        calculate_present_val(
            prin_amt=row.prin_amt,
            tenor=row.tenor,
            eir=row.eir,
            default_mob=row.default_mob,
            recovery_rate=row.recovery_rate,
            payment_freq=payment_freq,
            disc_rate=disc_rate
        )
        for row in gen_loan_info.itertuples(index=False)
    ]
    pv_loss = np.array(pv_loss)
    gen_loan_info['present_val'] = pv_loss[:, 0]
    gen_loan_info['loss_amt'] = pv_loss[:, 1]
    gen_loan_info['profit'] = gen_loan_info['present_val'] - gen_loan_info['prin_amt']


    num_groups = int(np.ceil(N / obs))
    labels = np.repeat(np.arange(1, num_groups + 1), obs)[:N]
    gen_loan_info['k_index'] = labels


    return gen_loan_info

## Run Data Simulation

In [365]:
def run_data_simulation(
    p_rating,
    p_prin_amt,
    p_tenor,
    p_existing,
    p_scenario,
    p_disc_rate,
    p_payment_freq,
    p_smallest_eir,
    p_largest_eir,
    p_port_quality=None
):
    """
    Parameters:
    p_rating: int
        Fixed credit rating for all new loans.
    p_prin_amt: float
        Fixed principal amount for all new loans.
    p_tenor: int
        Fixed tenor for all new loans.
    p_existing: int
        Number of existing loans to simulate.
    p_scenario: int
        Number of new loans to simulate.
    p_risk_tolerance: float
        Fraction representing risk tolerance.
    p_disc_rate: float
        Annual discount rate.
    p_payment_freq: int
        Number of payments per year.
    p_smallest_eir: float
        Smallest effective interest rate.
    p_largest_eir: float
        Largest effective interest rate.
    p_port_quality: str
        Portfolio quality indicator.

    Returns:
      k_bullet_df, p_k_bullet, p_k_minus, p_k_plus, avg_rating, obs_existing
    """

    obs_existing = p_existing
    k = p_scenario

    ids_new = np.arange(1, k + 1)
    new_df = simulate_new_loan(
        N_new=k,
        ids_new=ids_new,
        rating_input_new=p_rating,
        prin_input_new=p_prin_amt,
        tenor_input_new=p_tenor,
        payment_freq=p_payment_freq
    )

    N = obs_existing * k
    ids = np.arange(1, N + 1)
    curr_df = simulate_current_portfolio(
        N=N,
        ids=ids,
        obs=obs_existing,
        payment_freq=p_payment_freq,
        disc_rate=p_disc_rate,
        port_quality=p_port_quality
    )

    curr_agg = (
        curr_df
        .groupby("k_index", as_index=False)
        .agg(
            current_loss    = ("loss_amt", "sum"),
            current_prin_amt= ("prin_amt", "sum"),
            avg_rating      = ("credit_rating", "mean")
        )
    )

    sim = new_df.merge(curr_agg, on="k_index", how="inner")

    sim["loss_pct"] = (sim["current_loss"] / sim["current_prin_amt"])

    loss_pct_95 = np.percentile(sim["loss_pct"], 95)

    sim["portfolio_threshold"] = np.round(
        (sim["prin_amt"] + sim["current_prin_amt"]) * loss_pct_95,
        0
    )

    def pv_losses(eir):
        return np.array([
            calculate_present_val(
                prin_amt       = row.prin_amt,
                tenor          = row.tenor,
                default_mob    = row.default_mob,
                eir            = eir,
                recovery_rate  = row.recovery_rate,
                payment_freq   = p_payment_freq,
                disc_rate      = p_disc_rate
            )[0]
            for row in sim.itertuples()
        ])

    sim["pre_k_plus"]  = sim['prin_amt'] - pv_losses(p_smallest_eir) + sim["current_loss"]
    sim["pre_k_minus"] = sim['prin_amt'] - pv_losses(p_largest_eir) + sim["current_loss"]

    sim["k_plus"]  = (sim["pre_k_plus"]  < sim["portfolio_threshold"]).astype(int)
    sim["k_minus"] = (sim["pre_k_minus"] > sim["portfolio_threshold"]).astype(int)

    k_bullet = sim.query("k_plus == 0 and k_minus == 0").copy()

    k_bullet["m_minus"] = k_bullet["pre_k_plus"]  - k_bullet["portfolio_threshold"]
    k_bullet["m_plus"]  = k_bullet["portfolio_threshold"] - k_bullet["pre_k_minus"]

    total_k    = sim["k_index"].nunique()
    minus_k    = sim.loc[sim.k_minus==1,  "k_index"].nunique()
    plus_k     = sim.loc[sim.k_plus==1,   "k_index"].nunique()
    bullet_k   = k_bullet["k_index"].nunique()
    p_k_minus  = minus_k  / total_k
    p_k_plus   = plus_k   / total_k
    p_k_bullet = bullet_k / total_k

    avg_rating = np.round(k_bullet["avg_rating"].mean(), 2)

    print(f"obs_existing={obs_existing}:, "
            f"k_bullet={len(k_bullet)}, "
            f"p_k_minus={p_k_minus}, "
            f"p_k_bullet={p_k_bullet}, "
            f"p_k_plus={p_k_plus}, "
            f"avg_rating={avg_rating}")

    return k_bullet, p_k_bullet, p_k_minus, p_k_plus, avg_rating, obs_existing

## Run LPO VaR Model

In [434]:

def run_lpo_var_model(
    k_bullet,
    p_k_minus: float,
    p_k_bullet: float,
    avg_rating: float,
    smallest_eir: float,
    largest_eir: float,
    marginal_risk: float,
    payment_frequency: int,
    discount_rate: float,
    intercept: float,
    coef: float,
    alpha: float,
    epsilon: float = 1e-6,

):
    # ──────────────────────────────────────────────────────────────
    # 1.  Input pre-processing
    # ──────────────────────────────────────────────────────────────
    df = k_bullet.groupby('k_index', as_index=False).first()
    k_ids = df['k_index'].tolist()
    if not k_ids:
        raise ValueError("No scenarios found – check simulation parameters.")

    def col_dict(col):
        return df.set_index('k_index')[col].to_dict()

    # ──────────────────────────────────────────────────────────────
    # 2.  Model container & index sets
    # ──────────────────────────────────────────────────────────────
    m = ConcreteModel(name="LPO_VaR")
    m.k_index = Set(initialize=k_ids)

    # ──────────────────────────────────────────────────────────────
    # 3.  Parameters
    # ──────────────────────────────────────────────────────────────
    m.prin_amt            = Param(m.k_index, initialize=col_dict('prin_amt'))
    m.tenor               = Param(m.k_index, initialize=col_dict('tenor'))
    m.default_mob         = Param(m.k_index, initialize=col_dict('default_mob'))
    m.recovery_rate       = Param(m.k_index, initialize=col_dict('recovery_rate'))
    m.curr_loss           = Param(m.k_index, initialize=col_dict('current_loss'))
    m.portfolio_threshold = Param(m.k_index, initialize=col_dict('portfolio_threshold'))
    m.m_minus             = Param(m.k_index, initialize=col_dict('m_minus'))
    m.m_plus              = Param(m.k_index, initialize=col_dict('m_plus'))

    num_scenarios = len(k_ids)
    m.p_k_minus    = Param(initialize=p_k_minus)
    m.p            = Param(initialize=p_k_bullet)
    m.avg_rating   = Param(initialize=avg_rating)
    m.alpha        = Param(initialize=alpha)
    m.l            = Param(initialize=smallest_eir)
    m.u            = Param(initialize=largest_eir)
    m.marginal_risk = Param(initialize=marginal_risk)
    m.pay_freq     = Param(initialize=payment_frequency)
    m.disc_rate    = Param(initialize=discount_rate)
    m.intercept    = Param(initialize=intercept)
    m.coef        = Param(initialize=coef)
    m.epsilon      = Param(initialize=epsilon)

    # ──────────────────────────────────────────────────────────────
    # 4.  Decision variables
    # ──────────────────────────────────────────────────────────────
    m.x          = Var(bounds=(smallest_eir, largest_eir), within=NonNegativeReals)
    m.z_plus     = Var(m.k_index, within=NonNegativeReals)
    m.z_minus    = Var(m.k_index, within=NonNegativeReals)
    m.y_plus     = Var(m.k_index, within=NonNegativeReals)
    m.y_minus    = Var(m.k_index, within=NonNegativeReals)
    m.beta_plus  = Var(m.k_index, domain=Binary)
    m.beta_minus = Var(m.k_index, domain=Binary)

    m.k_t = Set(dimen=2, initialize=lambda me:
                [(k, t) for k in me.k_index for t in range(1, int(col_dict('tenor')[k]) + 1)]
                )

    # ──────────────────────────────────────────────────────────────
    # 5.  Discount factors & default indicators
    # ──────────────────────────────────────────────────────────────
    m.disc_factor = Param(
        m.k_t,
        initialize=lambda me, k, t: 1 / (1 + me.disc_rate / me.pay_freq) ** t
    )
    m.is_not_default = Param(
        m.k_t,
        within=Binary,
        initialize=lambda me, k, t:
            1 if (me.default_mob[k] == -1 or t < me.default_mob[k] - 3) else 0
    )

    # ──────────────────────────────────────────────────────────────
    # 6.  Cash-flow expressions
    # ──────────────────────────────────────────────────────────────


    def emi_rule(me, k):
        r = me.x / me.pay_freq
        n = me.tenor[k]
        return (1 / me.pay_freq) / (1 - (1/((1 + r) ** n)))
    m.emi_pmt = Expression(m.k_index, rule=emi_rule)

    def prin_pmt_rule(me, k, t):
        r = me.x / me.pay_freq
        n = me.tenor[k]
        # return me.emi_pmt[k] * ((1 + r) ** (t - 1 - n))
        return me.x * me.emi_pmt[k] * ((1 + r) ** (t - 1 - n))
    m.prin_pmt = Expression(m.k_t, rule=prin_pmt_rule)

    def recovery_rule(me, k):
        d = int(me.default_mob[k])
        if d == -1:
            return 0
        else:
            if d == 4:
                paid = 0
            else:
                paid = sum(me.prin_pmt[k, j] for j in range(1, d - 3))
            remaining = 1 - paid

            return remaining * me.recovery_rate[k]
    m.recovery_amt = Expression(m.k_index, rule=recovery_rule)

    m.present_val = Expression(m.k_index, rule=lambda me, k:
        me.prin_amt[k] * (
            sum(me.emi_pmt[k] * me.is_not_default[k, t] * me.disc_factor[k, t] * me.x
                for t in range(1, int(me.tenor[k]) + 1))
            + me.recovery_amt[k]
        )
    )

    m.net_value = Expression(m.k_index, rule=lambda me, k:
        me.present_val[k] - me.prin_amt[k]
    )

    # ──────────────────────────────────────────────────────────────
    # 7.  Objective (expected risk-weighted value)
    # ──────────────────────────────────────────────────────────────
    def g(me):
        return exp(me.intercept + me.coef * me.x) / (
               1 + exp(me.intercept + me.coef * me.x)
        )

    m.obj = Objective(
        rule=lambda me: g(me) * sum(me.net_value[k] for k in me.k_index)/num_scenarios,
        sense=maximize
    )

    # ──────────────────────────────────────────────────────────────
    # 8.  Constraints
    # ──────────────────────────────────────────────────────────────
    m.balance = Constraint(m.k_index, rule=lambda me, k:
        me.prin_amt[k] - me.present_val[k] + me.curr_loss[k]
        + me.z_plus[k] - me.z_minus[k] == me.portfolio_threshold[k]
    )

    m.z_bounds = ConstraintList()
    for k in m.k_index:
        m.z_bounds.add(m.epsilon * m.beta_plus[k] <= m.z_plus[k])
        m.z_bounds.add(m.z_plus[k] <= m.m_plus[k] * m.beta_plus[k])
        m.z_bounds.add(m.epsilon * m.beta_minus[k] <= m.z_minus[k])
        m.z_bounds.add(m.z_minus[k] <= m.m_minus[k] * m.beta_minus[k])
        m.z_bounds.add(m.beta_plus[k] + m.beta_minus[k] <= 1)

    # VaR probability constraint
    m.risk_constr = Constraint(rule=lambda me:
        sum(me.p * me.beta_minus[k] for k in me.k_index) + me.p_k_minus <= 1 - me.alpha
    )

    # McCormick envelopes
    m.mc = ConstraintList()
    # for k in k_ids:
    for k in m.k_index:
        # y_plus
        m.mc.add(m.y_plus[k] <= m.beta_plus[k] * m.u)
        m.mc.add(m.y_plus[k] >= m.x + m.beta_plus[k] * m.u - m.u)
        m.mc.add(m.y_plus[k] <= m.x)
        m.mc.add(m.y_plus[k] >= 0)
        # y_minus
        m.mc.add(m.y_minus[k] <= m.beta_minus[k] * m.u)
        m.mc.add(m.y_minus[k] >= m.x + m.beta_minus[k] * m.u - m.u)
        m.mc.add(m.y_minus[k] <= m.x)
        m.mc.add(m.y_minus[k] >= 0)

    # Chance constraint
    m.chance = Constraint(rule=lambda me:
        sum(me.p * (1 - me.beta_plus[k] - me.beta_minus[k])
            * (1 - me.recovery_amt[k] - me.marginal_risk)
            for k in me.k_index)
        - sum(me.p
            * sum(me.emi_pmt[k] * me.is_not_default[k, t] * me.disc_factor[k, t]
                for t in range(1, int(me.tenor[k]) + 1)) * (me.x - me.y_plus[k] - me.y_minus[k])
            for k in me.k_index)
        <= 0
    )

    m.decision_variables = {
        'x': m.x,
        'net_value': m.net_value,
        'present_val': m.present_val,
        'prin_pmt': m.prin_pmt,
        'tenor': m.tenor,
        'default_mob': m.default_mob,
        'recovery_rate': m.recovery_rate,
        'prin_amt': m.prin_amt,
        'curr_loss': m.curr_loss,
        'portfolio_threshold': m.portfolio_threshold,
        'emi_pmt': m.emi_pmt,
        'recovery_amt:': m.recovery_amt,
        'z_plus': m.z_plus,
        'z_minus': m.z_minus,
        'y_plus': m.y_plus,
        'y_minus': m.y_minus,
        'beta_plus': m.beta_plus,
        'beta_minus': m.beta_minus,
        'm_plus': m.m_plus,
        'm_minus': m.m_minus
    }

    return m

## Run Pricing Model

In [398]:
def run_pricing_simulation(p_rating
                           ,p_prin_amt
                           ,p_tenor
                           ,p_existing
                           ,p_scenario
                           ,p_smallest_eir
                           ,p_largest_eir
                           ,p_marginal_risk
                           ,p_disc_rate
                           ,p_payment_freq
                           ,p_alpha
                           ,p_solver
                           ,p_port_quality=None
                           ,p_show_log='n'
                           ,p_seed=None):

    np.random.seed(p_seed)  # reproducibility

    start_time = time.time()


    data_simulation = run_data_simulation(p_rating=p_rating
                                      ,p_prin_amt=p_prin_amt
                                      ,p_tenor=p_tenor
                                      ,p_existing=p_existing
                                      ,p_scenario=p_scenario
                                      ,p_smallest_eir=p_smallest_eir
                                      ,p_largest_eir=p_largest_eir
                                      ,p_disc_rate=p_disc_rate
                                      ,p_payment_freq=p_payment_freq
                                      ,p_port_quality=p_port_quality)
    if data_simulation[0].empty:
        print(f"k_bullet is empty for rating: {p_rating}. Skipping to next rating.")
        return None, None, None


    model = run_lpo_var_model(k_bullet=data_simulation[0]
                          , p_k_bullet=data_simulation[1]
                          , p_k_minus=data_simulation[2]
                          , avg_rating=data_simulation[4]
                          , smallest_eir=p_smallest_eir
                          , largest_eir=p_largest_eir
                          , marginal_risk=p_marginal_risk
                          , payment_frequency=p_payment_freq
                          , discount_rate=p_disc_rate
                          , intercept=intercept
                          , coef=coef
                          , alpha=p_alpha)

    solver = SolverFactory(p_solver)
    results = solver.solve(model, tee=False)

    end_time = time.time()
    elapsed_time = np.round(end_time - start_time, 0)

    if results.solver.termination_condition == TerminationCondition.infeasible:
        optimal_x = None
        optimal_obj = None
    else:
        optimal_x = np.round(value(model.x), 4)
        optimal_obj = np.round(value(model.obj), 4)


    avg_rating = np.round(value(model.avg_rating), 2)

    if p_show_log == 'y':

        for name, var in model.decision_variables.items():
            print(f"Variable: {name}")
            if hasattr(var, 'items'):
                for k, v in var.items():
                    print(f"  {k}: {value(v)}")
            else:
                print(f"  {var.name}: {value(var)}")

    return optimal_x, optimal_obj, avg_rating, elapsed_time

# SIMULATION

In [465]:
run_pricing_simulation(p_rating=4
                        ,p_prin_amt=30
                        ,p_tenor=12
                        ,p_existing=50
                        ,p_scenario=500
                        ,p_marginal_risk=0.05
                        ,p_smallest_eir=0.18
                        ,p_largest_eir=0.50
                        ,p_alpha=0.90
                        ,p_disc_rate=0.09
                        ,p_payment_freq=12
                        ,p_solver='couenne'
                        # ,p_solver='bonmin'
                        ,p_port_quality=None
                        ,p_show_log='n'
                        ,p_seed=None
                    )

obs_existing=50:, k_bullet=8, p_k_minus=0.03, p_k_bullet=0.016, p_k_plus=0.954, avg_rating=5.66


  - termination condition: infeasible
  - message from solver: Couenne (/tmp/tmp7xm6tj02.pyomo.nl Jun  7 2023)\x3a Infeasible


(None, None, np.float64(5.66), np.float64(5.0))

In [470]:
run_pricing_simulation(p_rating=4
                        ,p_prin_amt=30
                        ,p_tenor=12
                        ,p_existing=50
                        ,p_scenario=500
                        ,p_marginal_risk=0.05
                        ,p_smallest_eir=0.18
                        ,p_largest_eir=0.50
                        ,p_alpha=0.90
                        ,p_disc_rate=0.09
                        ,p_payment_freq=12
                        # ,p_solver='couenne'
                        ,p_solver='bonmin'
                        ,p_port_quality=None
                        ,p_show_log='n'
                        ,p_seed=None
                    )

obs_existing=50:, k_bullet=3, p_k_minus=0.042, p_k_bullet=0.006, p_k_plus=0.952, avg_rating=5.85


(np.float64(0.2991), np.float64(1.2068), np.float64(5.85), np.float64(5.0))

In [498]:
import pandas as pd

# prepare a list to hold each run’s summary
summaries = []

# for disc_Rate in range(1, 11):
for existing in [50,100,150,200,250,300]:
    result = run_pricing_simulation(
            p_rating      = 4,
            p_prin_amt    = 30,
            p_tenor       = 12,
            p_existing    = existing,
            p_scenario    = 2500, #This was scenario and it caused the error
            p_marginal_risk = 0.05,
            p_smallest_eir  = 0.18,
            p_largest_eir   = 0.50,
            p_alpha         = 0.90,
            p_disc_rate     = 0.09,
            p_payment_freq  = 12,
            p_solver       = 'bonmin',
            p_port_quality = None,
            p_show_log     = 'n',
            p_seed         = None
        )
    #Check if the result has 4 values and add to the list
    if result and len(result) == 4:
        optimal_x, optimal_obj, avg_rating, elapsed_time = result
        summaries.append({
            'scenario'        : existing,
            'existing'      : optimal_x,
            'optimal_obj'    : optimal_obj,
            'avg_rating'    : avg_rating,
            'elapsed_time'  : elapsed_time
        })
    else:
        print(f"Skipping scenario {existing} due to empty k_bullet or invalid result")


# combine into one DataFrame
results_df = pd.DataFrame(summaries)

# if you like, save or inspect
print(results_df)
# results_df.to_csv('pricing_sweep.csv', index=False)

obs_existing=50:, k_bullet=21, p_k_minus=0.0356, p_k_bullet=0.0084, p_k_plus=0.956, avg_rating=5.8
obs_existing=100:, k_bullet=17, p_k_minus=0.0424, p_k_bullet=0.0068, p_k_plus=0.9508, avg_rating=5.65
obs_existing=150:, k_bullet=20, p_k_minus=0.0416, p_k_bullet=0.008, p_k_plus=0.9504, avg_rating=5.64
obs_existing=200:, k_bullet=12, p_k_minus=0.042, p_k_bullet=0.0048, p_k_plus=0.9532, avg_rating=5.64
obs_existing=250:, k_bullet=17, p_k_minus=0.0432, p_k_bullet=0.0068, p_k_plus=0.95, avg_rating=5.69
obs_existing=300:, k_bullet=13, p_k_minus=0.0416, p_k_bullet=0.0052, p_k_plus=0.9532, avg_rating=5.65
   scenario  existing  optimal_obj  avg_rating  elapsed_time
0        50    0.4486       0.8706        5.80          29.0
1       100    0.2977       1.2108        5.65          49.0
2       150    0.3414       0.8551        5.64          74.0
3       200    0.2984       1.1276        5.64          97.0
4       250    0.3912       1.0632        5.69         122.0
5       300    0.2991       1

# OUT OF SAMPLE TEST

## Out of sample (Actual data)

In [500]:
url_out_of_sample = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
54410a81dfc4e0d313bad3de09ffaf91d6660a62/data/out_of_sample_dataset20.csv"

out_of_sample = pd.read_csv(url_out_of_sample)

out_of_sample.head()

,rating,default_mob,tenor,eir,prin_amt,lgd,recovery_rate
0,10,-1,24,0.38,15,-1.0,1.0
1,9,-1,48,0.38,15,-1.0,1.0
2,1,-1,24,0.31,100,-1.0,1.0
3,6,7,12,0.38,15,-1.0,1.0
4,4,-1,24,0.38,20,-1.0,1.0


In [502]:
out_of_sample['act_present_val'] = out_of_sample.apply(lambda row: calculate_present_val(
    prin_amt=row['prin_amt'],
    tenor=row['tenor'],
    eir=row['eir'],
    default_mob=row['default_mob'],
    recovery_rate=row['recovery_rate'],
    payment_freq=12,
    disc_rate=0.27
)[0], axis=1)

out_of_sample['act_profit'] = out_of_sample['act_present_val'] - out_of_sample['prin_amt']

out_of_sample.head()

,rating,default_mob,tenor,eir,prin_amt,lgd,recovery_rate,act_present_val,act_profit
0,10,-1,24,0.38,15,-1.0,1.0,17.0,2.0
1,9,-1,48,0.38,15,-1.0,1.0,18.0,3.0
2,1,-1,24,0.31,100,-1.0,1.0,104.0,4.0
3,6,7,12,0.38,15,-1.0,1.0,16.0,1.0
4,4,-1,24,0.38,20,-1.0,1.0,22.0,2.0


In [504]:
average_profit = out_of_sample['act_profit'].mean()
print(f"The average actual profit is: {average_profit}")

The average actual profit is: -16.65


In [505]:
average_eir = out_of_sample['eir'].mean()
print(f"The average actual eir is: {average_eir}")

The average actual eir is: 0.36299999999999993


## Out of sample (Simulation data)

In [507]:
results_test = []

for index, row in out_of_sample.iterrows():
    result = run_pricing_simulation(
        p_rating=row['rating'],
        p_prin_amt=row['prin_amt'],
        p_tenor=row['tenor'],
        p_existing=100,
        p_scenario=2500,
        p_marginal_risk=0.05,
        p_smallest_eir=0.18,
        p_largest_eir=0.50,
        p_alpha=0.90,
        p_disc_rate=0.09,
        p_payment_freq=12,
        p_solver='bonmin',
        p_port_quality=None,
        p_show_log='n',
        p_seed=None
    )
    results_test.append(result)


cleaned_results = [result for result in results_test if result is not None]
df_results_test = pd.DataFrame(cleaned_results, columns=['Opt_Price', 'Opt_Obj', 'Avg_Rating', 'Solving_Time'])
df_results_test = pd.concat([out_of_sample, df_results_test], axis=1)
df_results_test.head()

df_results_test.to_csv('out_of_sample_opt20.csv', index=False)
# files.download('out_of_sample_opt20.csv')

obs_existing=100:, k_bullet=17, p_k_minus=0.0456, p_k_bullet=0.0068, p_k_plus=0.9476, avg_rating=5.62
obs_existing=100:, k_bullet=22, p_k_minus=0.0456, p_k_bullet=0.0088, p_k_plus=0.9456, avg_rating=5.59


  - termination condition: infeasible
  - message from solver: bonmin\x3a Infeasible problem


obs_existing=100:, k_bullet=43, p_k_minus=0.0128, p_k_bullet=0.0172, p_k_plus=0.97, avg_rating=5.77
obs_existing=100:, k_bullet=3, p_k_minus=0.046, p_k_bullet=0.0012, p_k_plus=0.9528, avg_rating=5.63
obs_existing=100:, k_bullet=25, p_k_minus=0.0376, p_k_bullet=0.01, p_k_plus=0.9524, avg_rating=5.66
obs_existing=100:, k_bullet=21, p_k_minus=0.0464, p_k_bullet=0.0084, p_k_plus=0.9452, avg_rating=5.71


  - termination condition: infeasible
  - message from solver: bonmin\x3a Infeasible problem


obs_existing=100:, k_bullet=56, p_k_minus=0.018, p_k_bullet=0.0224, p_k_plus=0.9596, avg_rating=5.7
obs_existing=100:, k_bullet=61, p_k_minus=0.0116, p_k_bullet=0.0244, p_k_plus=0.964, avg_rating=5.67
obs_existing=100:, k_bullet=45, p_k_minus=0.0276, p_k_bullet=0.018, p_k_plus=0.9544, avg_rating=5.68
obs_existing=100:, k_bullet=44, p_k_minus=0.0236, p_k_bullet=0.0176, p_k_plus=0.9588, avg_rating=5.68
obs_existing=100:, k_bullet=61, p_k_minus=0.0164, p_k_bullet=0.0244, p_k_plus=0.9592, avg_rating=5.7
obs_existing=100:, k_bullet=48, p_k_minus=0.0236, p_k_bullet=0.0192, p_k_plus=0.9572, avg_rating=5.64
obs_existing=100:, k_bullet=69, p_k_minus=0.0504, p_k_bullet=0.0276, p_k_plus=0.922, avg_rating=5.68


  - termination condition: infeasible
  - message from solver: bonmin\x3a Infeasible problem


obs_existing=100:, k_bullet=19, p_k_minus=0.04, p_k_bullet=0.0076, p_k_plus=0.9524, avg_rating=5.7
obs_existing=100:, k_bullet=36, p_k_minus=0.0252, p_k_bullet=0.0144, p_k_plus=0.9604, avg_rating=5.66
obs_existing=100:, k_bullet=23, p_k_minus=0.0428, p_k_bullet=0.0092, p_k_plus=0.948, avg_rating=5.69
obs_existing=100:, k_bullet=51, p_k_minus=0.0108, p_k_bullet=0.0204, p_k_plus=0.9688, avg_rating=5.69
obs_existing=100:, k_bullet=40, p_k_minus=0.036, p_k_bullet=0.016, p_k_plus=0.948, avg_rating=5.73
obs_existing=100:, k_bullet=51, p_k_minus=0.0268, p_k_bullet=0.0204, p_k_plus=0.9528, avg_rating=5.67
obs_existing=100:, k_bullet=37, p_k_minus=0.0212, p_k_bullet=0.0148, p_k_plus=0.964, avg_rating=5.69


In [513]:
run_pricing_simulation(
        p_rating=8,
        p_prin_amt=70,
        p_tenor=48,
        p_existing=100,
        p_scenario=2500,
        p_marginal_risk=0.05,
        p_smallest_eir=0.18,
        p_largest_eir=0.50,
        p_alpha=0.90,
        p_disc_rate=0.09,
        p_payment_freq=12,
        p_solver='bonmin',
        p_port_quality=None,
        p_show_log='n',
        p_seed=None
    )

obs_existing=100:, k_bullet=56, p_k_minus=0.0204, p_k_bullet=0.0224, p_k_plus=0.9572, avg_rating=5.62


(np.float64(0.4923), np.float64(4.6174), np.float64(5.62), np.float64(68.0))

In [514]:
url_out_of_sample_opt = "https://raw.githubusercontent.com/nlamduy/UEH_MS_Thesis/\
09821225411a68ad08dfc27c3435e778a9e4fce0/data/out_of_sample_opt20_final.csv"

out_of_sample_opt = pd.read_csv(url_out_of_sample_opt)

out_of_sample_opt['simul_present_val'] = out_of_sample_opt.apply(lambda row: calculate_present_val(
    prin_amt=row['prin_amt'],
    tenor=row['tenor'],
    eir=row['Opt_Price'],
    default_mob=row['default_mob'],
    recovery_rate=row['recovery_rate'],
    payment_freq=12,
    disc_rate=0.09
)[0], axis=1)

out_of_sample_opt['opt_profit'] = out_of_sample_opt['simul_present_val'] - out_of_sample_opt['prin_amt']

out_of_sample_opt.head()

,index,rating,default_mob,tenor,eir,prin_amt,lgd,recovery_rate,act_present_val,act_profit,Opt_Price,Opt_Obj,Avg_Rating,Solving_Time,simul_present_val,opt_profit
0,0,10,-1,24,0.38,15,-1,1,17,2,0.4457,0.5193,5.62,50,21.0,6.0
1,1,9,-1,48,0.38,15,-1,1,18,3,0.4689,1.3143,5.63,56,28.0,13.0
2,2,1,-1,24,0.31,100,-1,1,104,4,0.4507,5.8691,5.77,62,140.0,40.0
3,3,6,7,12,0.38,15,-1,1,16,1,0.2991,0.6034,5.63,50,16.0,1.0
4,4,4,-1,24,0.38,20,-1,1,22,2,0.4507,1.1058,5.66,55,28.0,8.0


In [515]:
average_opt_profit = out_of_sample_opt['opt_profit'].mean()
print(f"The average optimal profit is: {average_opt_profit}")

The average optimal profit is: -5.5


In [516]:
average_opt_eir = out_of_sample_opt['Opt_Price'].mean()
print(f"The average optimal eir is: {average_opt_eir}")

The average optimal eir is: 0.442105
